In [1]:
import pandas as pd

activity_df = pd.read_csv('data/clean/activity.csv')
drug_df = pd.read_csv('data/clean/drug.csv')
glucose_df = pd.read_csv('data/clean/glucose.csv')
food_df = pd.read_csv('data/clean/food.csv')

In [6]:
glucose_df['datetime'] = pd.to_datetime(glucose_df['datetime'])
activity_df['start_datetime'] = pd.to_datetime(activity_df['start_datetime'])
activity_df['end_datetime'] = pd.to_datetime(activity_df['end_datetime'])
food_df['datetime'] = pd.to_datetime(food_df['datetime'])
drug_df['datetime'] = pd.to_datetime(drug_df['datetime'])

In [3]:
import plotly.express as px

fig = px.line(glucose_df, x='datetime', y='glucose_mg_dl', title='Glucose Level Over Time')
fig.update_layout(xaxis=dict(title='Datetime'), yaxis=dict(title='Glucose Level (mg/dL)'))


fig.show()

In [7]:
import plotly.graph_objects as go

def plot_day(glucose_df: pd.DataFrame, food_df: pd.DataFrame, activity_df: pd.DataFrame, drug_df: pd.DataFrame, date: str) -> go.Figure:
    # Filter glucose data for the given date
    date_as_datetime = pd.to_datetime(date)
    glucose_data = glucose_df[(glucose_df['datetime'].dt.date == date_as_datetime.date())]
    food_data = food_df[(food_df['datetime'].dt.date == date_as_datetime.date())]
    drug_data = drug_df[(drug_df['datetime'].dt.date == date_as_datetime.date())]

    activities_from_this_day = activity_df[(activity_df['start_datetime'].dt.date == date_as_datetime.date())]

    # Create a figure
    fig = go.Figure()

    # Add glucose level trace
    fig.add_trace(go.Scatter(x=glucose_data['datetime'], y=glucose_data['glucose_mg_dl'], name='Glucose Level'))

    for index, row in drug_data.iterrows():
        fig.add_annotation(x=row['datetime'], y=glucose_data['glucose_mg_dl'].max() + 120, 
                           text=f"{row['name']} ({row['dosage']})", showarrow=True, arrowhead=2, ax=0, ay=-60, textangle=-45, bgcolor='lightyellow')

    for index, row in food_data.iterrows():
        if len(row["name"]) > 20:
            chunks = row["name"].split(" ")
            chunks = [" ".join(chunks[i:i+5]) for i in range(0, len(chunks), 5)]
            title = "<br>".join(chunks)
        else:
            title = row["name"]
        fig.add_annotation(x=row['datetime'], y=glucose_data['glucose_mg_dl'].max() + 60, 
                           text=title, showarrow=True, arrowhead=2, ax=0, ay=-40, textangle=-45)

    # Add activity intervals
    for index, row in activities_from_this_day.iterrows():
        fig.add_shape(type="rect",
                      x0=row['start_datetime'], x1=row['end_datetime'], y0=glucose_data['glucose_mg_dl'].min(), y1=glucose_data['glucose_mg_dl'].max(),
                      line=dict(color="RoyalBlue", width=2, dash="dashdot"),
                      fillcolor="LightSkyBlue", opacity=0.2)
        
        activity_text = f"{row['title']}<br>Max HR: {row['max_hr']}<br>Avg HR: {row['avg_hr']}"
        fig.add_annotation(x=row['start_datetime'] + (row['end_datetime'] - row['start_datetime']) / 2, 
                           y=glucose_data['glucose_mg_dl'].max() - 10, 
                           text=activity_text, showarrow=False, yshift=10)

    # Update layout
    fig.update_layout(title=f"Glucose Level for {date}",
                      xaxis=dict(title="Time"),
                      yaxis=dict(title="Glucose Level (mg/dL)"))

    return fig

fig = plot_day(glucose_df, food_df, activity_df, drug_df, '2024-06-13')
fig.show()

In [8]:
# for each unique, sorted day in glucose_df, plot the day
for date in glucose_df['datetime'].dt.date.unique()[:-3]:
    plot_day(glucose_df, food_df, activity_df, drug_df, str(date)).show()

In [9]:
import plotly.io as pio
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
import io

def plots_to_pdf(output_filename):
    figures = []

    for date in glucose_df['datetime'].dt.date.unique()[:-3]:
        fig = plot_day(glucose_df, food_df, activity_df, drug_df, str(date))
        figures.append(fig)

    pdf_canvas = canvas.Canvas(output_filename, pagesize=letter)
    
    w, h = 650, 500
    for fig in figures:
        img_bytes = io.BytesIO()
        pio.write_image(fig, img_bytes, format='png', width=w, height=h)
        img_bytes.seek(0)
        
        # Create an ImageReader object
        img = ImageReader(img_bytes)
        
        # Calculate the position to center the image on the page
        page_width, page_height = letter
        img_width, img_height = w, h
        x_position = (page_width - img_width) / 2
        y_position = (page_height - img_height) / 2
        
        # Draw the image on the canvas
        pdf_canvas.drawImage(img, x_position, y_position, width=img_width, height=img_height)
        pdf_canvas.showPage()
    
    # Save the PDF
    pdf_canvas.save()

plots_to_pdf('glucose_reports.pdf')